## Computational Theory Assignment

In [1]:
import numpy as np

## Problem 1: Binary Words and Operations

In [2]:
def parity(x, y, z):
    # bitwise XOR
    ## Example 1 0 1, if there are odd number of 1s the result is 1
    ## if there are even number of 1s the result is 0
    ## result of 1 0 1 would be 0 as there are an even number of 1s
    return x ^ y ^ z

print(bin(parity(0b1100, 0b1111, 0b1000)))


0b1011


The `numpy.array` function ([see official documentation](https://numpy.org/doc/stable/reference/generated/numpy.array.html)) creates an array from a list or other data structure.

In [3]:
a = np.array([0, 1, 0, 1])
b = np.array([1, 1, 0, 1])
c = np.array([0, 0, 0, 1])

print(parity(a, b , c))

[1 0 0 1]


In [4]:
def Ch(x, y ,z):
    ## If a bit in x is 1, choose the bit from y
    ## If a bit in x is 0, choose the bit from z
    return (x & y) ^ (~x & z)

print(bin(Ch(0b1001, 0b1100, 0b1010)))

0b1010


In [5]:
def Maj(x, y ,z):
    ## Each bit in the output equals the majority(maj) vote of the bits of x,y,z
    ## example:
    # 1001
    # 1100
    # 1010
    # result -> 1000

    # Convert inputs to 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return (x & y) ^ (x & z) ^ (y & z)

print(bin(Maj(0b1001, 0b1100, 0b1010)))

0b1000


In [6]:
def Sigma0(x):
    ## Σ0(x) = ROTR^2(x) ⊕ ROTR^13(x) ⊕ ROTR^22(x)
    ## ROTR means to rotate right ROTR^2 mean rotate 2 bits from the right
    ## i.e 1011 -> 1110
    rotr2  = ((x >> 2)  | (x << (32 - 2)))  & 0xffffffff
    rotr13 = ((x >> 13) | (x << (32 - 13))) & 0xffffffff
    rotr22 = ((x >> 22) | (x << (32 - 22))) & 0xffffffff
    return rotr2 ^ rotr13 ^ rotr22

x = 0x12345678
result = Sigma0(x)
print(hex(result))
print(bin(result))

0x66146474
0b1100110000101000110010001110100


In [7]:
def Sigma1(x):
    ## Σ1(x) = ROTR^6(x) ⊕ ROTR^11(x) ⊕ ROTR^25(x)
    ## Rotate x right by 6 bits
    ## Rotate x right by 11 bits
    ## Rotate x right by 25 bits
    ## XOR (⊕) the three results together
    rotr6  = ((x >> 6)  | (x << (32 - 6)))  & 0xffffffff
    rotr11 = ((x >> 11) | (x << (32 - 11))) & 0xffffffff
    rotr25 = ((x >> 25) | (x << (32 - 25))) & 0xffffffff
    return rotr6 ^ rotr11 ^ rotr25

In [8]:
def sigma0(x):
    ## σ0(x) = ROTR^7(x) ⊕ ROTR^18(x) ⊕ SHR^3(x)

    ## Rotate x right by 7 bits
    ## Rotate x right by 18 bits
    ## Shift x to the right by 3 bits
    rotr7  = ((x >> 7)  | (x << (32 - 7)))  & 0xffffffff
    rotr18 = ((x >> 18) | (x << (32 - 18))) & 0xffffffff
    shr3   = (x >> 3)
    return rotr7 ^ rotr18 ^ shr3

In [9]:
def sigma1(x):
    ## σ1(x) = ROTR^17(x) ⊕ ROTR^19(x) ⊕ SHR^10(x)
    rotr17 = ((x >> 17) | (x << (32 - 17))) & 0xffffffff
    rotr19 = ((x >> 19) | (x << (32 - 19))) & 0xffffffff
    shr10  = (x >> 10)
    return rotr17 ^ rotr19 ^ shr10

In [10]:
x = 0x12345678
print(hex(Sigma0(x)))
print(hex(Sigma1(x)))
print(hex(sigma0(x)))
print(hex(sigma1(x)))

0x66146474
0x3561abda
0xe7fce6ee
0xa1f78649


## Problem 2: Fractional Parts of Cube Roots

The `numpy.array` function ([see official documentation](https://numpy.org/doc/stable/reference/generated/numpy.array.html)) creates an array from a list or other data structure.

In [11]:
def primes(n):
    ## storing the first n prime numbers
    
    prime_list = []
    ##smallest prime number and is the current number testing for primality
    current_number = 2
    ## checking numbers until we found n prime numbers and tracks how many primes we've found so far
    while len(prime_list) < n:
        ## start by assuming current_number is prime
        is_prime = True

        for p in prime_list:
            ## if loop stops when p^2 is bigger than current_number because any composite number has to have a factor of less than or equal to its root
            if p * p > current_number:
                break
            ## % is the modulo operator, if current_number is divisble by p then it is not a prime number
            if current_number % p == 0:
                is_prime = False
                break
            
        ## if current_number is prime store it into a np.array
        if is_prime:
            prime_list.append(current_number)

        current_number += 1
        ## unint64 makes sure all values are non negative
    return np.array(prime_list,dtype=np.uint64)


### Calculates the cube roots of the first 64 prime numbers

The `numpy.cbrt` function ([see official documentation](https://numpy.org/doc/stable/reference/generated/numpy.cbrt.html)) returns the cube root of an array, element wise.

In [12]:
p = primes(64)
cube_roots = np.cbrt(p)

### Extracts the first 32 bits of the fractional part

In [13]:
##np.floor removes the interger part
##multiplying by 2**32 shifts the fraction part into a 32-bit integer
fractional_parts = cube_roots - np.floor(cube_roots)
constants = (fractional_parts * 2**32).astype(np.uint32)

In [14]:
for i, c in enumerate(constants):
    print(f"K[{i:2}] = 0x{c:08x}")

K[ 0] = 0x428a2f98
K[ 1] = 0x71374491
K[ 2] = 0xb5c0fbcf
K[ 3] = 0xe9b5dba5
K[ 4] = 0x3956c25b
K[ 5] = 0x59f111f1
K[ 6] = 0x923f82a4
K[ 7] = 0xab1c5ed5
K[ 8] = 0xd807aa98
K[ 9] = 0x12835b01
K[10] = 0x243185be
K[11] = 0x550c7dc3
K[12] = 0x72be5d74
K[13] = 0x80deb1fe
K[14] = 0x9bdc06a7
K[15] = 0xc19bf174
K[16] = 0xe49b69c1
K[17] = 0xefbe4786
K[18] = 0x0fc19dc6
K[19] = 0x240ca1cc
K[20] = 0x2de92c6f
K[21] = 0x4a7484aa
K[22] = 0x5cb0a9dc
K[23] = 0x76f988da
K[24] = 0x983e5152
K[25] = 0xa831c66d
K[26] = 0xb00327c8
K[27] = 0xbf597fc7
K[28] = 0xc6e00bf3
K[29] = 0xd5a79147
K[30] = 0x06ca6351
K[31] = 0x14292967
K[32] = 0x27b70a85
K[33] = 0x2e1b2138
K[34] = 0x4d2c6dfc
K[35] = 0x53380d13
K[36] = 0x650a7354
K[37] = 0x766a0abb
K[38] = 0x81c2c92e
K[39] = 0x92722c85
K[40] = 0xa2bfe8a1
K[41] = 0xa81a664b
K[42] = 0xc24b8b70
K[43] = 0xc76c51a3
K[44] = 0xd192e819
K[45] = 0xd6990624
K[46] = 0xf40e3585
K[47] = 0x106aa070
K[48] = 0x19a4c116
K[49] = 0x1e376c08
K[50] = 0x2748774c
K[51] = 0x34b0bcb5
K[52] = 0x39

## Problem 3: Padding


Section 5.1.1 — Padding the Message
“The message is padded by appending a single ‘1’ bit, followed by enough ‘0’ bits so that the length of the padded message is congruent to 448 modulo 512, and then appending the 64-bit representation of the message length.”

In [15]:
def block_parse(msg: bytes):
    msg_len = len(msg) * 8  ## length in bits

    padded = msg + b'\x80'  ## '1' bit + 7 zeros
    ## keep adding zeroes until the length is 448 mod 512
    while (len(padded) * 8) % 512 != 448:
        padded += b'\x00'

    padded += msg_len.to_bytes(8, byteorder='big')

    for i in range(0, len(padded), 64):
        yield padded[i:i + 64]


In [16]:
def test_message(msg):
    print("Message length:", len(msg), "bytes")

    count = 0
    for block in block_parse(msg):
        print(" Block", count)
        print("  Size:", len(block), "bytes")
        print("  Last 8 bytes:", block[-8:].hex())
        count += 1

    print("Total blocks:", count)
    print("-" * 40)


In [17]:
test_message(b"")


Message length: 0 bytes
 Block 0
  Size: 64 bytes
  Last 8 bytes: 0000000000000000
Total blocks: 1
----------------------------------------


In [18]:
test_message(b"abc")


Message length: 3 bytes
 Block 0
  Size: 64 bytes
  Last 8 bytes: 0000000000000018
Total blocks: 1
----------------------------------------


In [19]:
test_message(b"a" * 56)


Message length: 56 bytes
 Block 0
  Size: 64 bytes
  Last 8 bytes: 8000000000000000
 Block 1
  Size: 64 bytes
  Last 8 bytes: 00000000000001c0
Total blocks: 2
----------------------------------------


## Problem 4: Hashes

The SHA-256 hash computation processes one 512-bit message block and updates
the current hash value. It is defined in Section 6.2.2.


In [20]:
def hash(current, block):
    ## create the message schedule array W[0..63]
    W = np.zeros(64, dtype=np.uint32)

    ## read the first 16 words directly from the message block (big-endian)
    for t in range(16):
        W[t] = np.uint32(
            int.from_bytes(block[t*4:(t+1)*4], 'big')
        )

    ## extend the first 16 words into the remaining 48 words
    for t in range(16, 64):
        W[t] = np.uint32(
            sigma1(W[t-2]) +
            W[t-7] +
            sigma0(W[t-15]) +
            W[t-16]
        )

    ## initialise working variables from the current hash value
    a = np.uint32(current[0])
    b = np.uint32(current[1])
    c = np.uint32(current[2])
    d = np.uint32(current[3])
    e = np.uint32(current[4])
    f = np.uint32(current[5])
    g = np.uint32(current[6])
    h = np.uint32(current[7])

    ## main loop (64 rounds)
    for t in range(64):
        ## T1 = h + Sigma1(e) + Ch(e,f,g) + K[t] + W[t]
        T1 = np.uint32(
            h +
            Sigma1(e) +
            Ch(e, f, g) +
            constants[t] +
            W[t]
        )
        ## T2 = Sigma0(a) + Maj(a,b,c)
        T2 = np.uint32(
            Sigma0(a) +
            Maj(a, b, c)
        )

        h = g
        g = f
        f = e
        e = np.uint32(d + T1)
        d = c
        c = b
        b = a
        a = np.uint32(T1 + T2)

    ## compute the next hash value
    return (
        np.uint32(current[0] + a),
        np.uint32(current[1] + b),
        np.uint32(current[2] + c),
        np.uint32(current[3] + d),
        np.uint32(current[4] + e),
        np.uint32(current[5] + f),
        np.uint32(current[6] + g),
        np.uint32(current[7] + h),
    )


### Verification

The implementation was verified using standard SHA-256 test vectors.
For the input message "abc", the computed hash matches the reference
value published by NIST and the output of Python’s hashlib library.

In [21]:
H0 = (
    0x6a09e667,
    0xbb67ae85,
    0x3c6ef372,
    0xa54ff53a,
    0x510e527f,
    0x9b05688c,
    0x1f83d9ab,
    0x5be0cd19,
)

msg = b"abc"
blocks = list(block_parse(msg))

print(len(blocks))      ## should be 1
print(len(blocks[0]))   ## should be 64

H = H0
H = hash(H, blocks[0])

digest = ''.join(f'{int(x):08x}' for x in H)
print(digest)



1
64
ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad


/tmp/ipykernel_12795/2265186251.py:14: RuntimeWarning: overflow encountered in scalar add
  sigma1(W[t-2]) +
/tmp/ipykernel_12795/2265186251.py:34: RuntimeWarning: overflow encountered in scalar add
  h +
/tmp/ipykernel_12795/2265186251.py:42: RuntimeWarning: overflow encountered in scalar add
  Sigma0(a) +
/tmp/ipykernel_12795/2265186251.py:49: RuntimeWarning: overflow encountered in scalar add
  e = np.uint32(d + T1)
/tmp/ipykernel_12795/2265186251.py:53: RuntimeWarning: overflow encountered in scalar add
  a = np.uint32(T1 + T2)
/tmp/ipykernel_12795/2265186251.py:58: RuntimeWarning: overflow encountered in scalar add
  np.uint32(current[1] + b),
/tmp/ipykernel_12795/2265186251.py:60: RuntimeWarning: overflow encountered in scalar add
  np.uint32(current[3] + d),
/tmp/ipykernel_12795/2265186251.py:62: RuntimeWarning: overflow encountered in scalar add
  np.uint32(current[5] + f),


## Problem 5: Passwords

A dictionary attack was used using a list of common passwords. Each
password was encoded using UTF8, hashed using SHA256 and
compared against the given hash values. Because the hashes were unsalted
and generated using a single SHA256 iteration the passwords
given were quickly recovered.

In [ ]:
hashes = {
    "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8",
    "873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34",
    "b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342",
}


In [ ]:
import hashlib

## Small dictionary of common passwords
dictionary = [
    "password",
    "123456",
    "123456789",
    "qwerty",
    "abc123",
    "letmein",
    "admin",
    "welcome",
    "password1",
    "iloveyou"
]

def sha256(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()

for word in dictionary:
    h = sha256(word)
    if h in hashes:
        print(f"Found match: {word} → {h}")


Found match: password → 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8


SHA256 is designed to be fast, which is bad for passwords. Instead you should use a slow hashing algorithim that takes more time and resources to compute each hash this reduces the number of password guesses an attacker can perform per second, making dictionary and brute force attacks impractical.